In [16]:
import os
import streamlit as st
import tempfile
from dotenv import load_dotenv

from openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document

In [2]:
#Chroma tenant 오류 방지 위한 코드
import chromadb
chromadb.api.client.SharedSystemClient.clear_system_cache()

#오픈AI API 키 설정
load_dotenv()  # 현재 경로의 .env 로드
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
if not os.environ['OPENAI_API_KEY']:
    raise ValueError('OPENAI_API_KEY not found in environment. set it in .env or env vars')

## 1. 문서 불러오기

In [3]:
import tempfile

In [4]:
file_path = '../data/langchainThon/cv/경력기술서_천승우.docx'
# 파일을 바이너리 모드로 읽어서 _file처럼 흉내내기
class DummyFile:
    def __init__(self, path):
        self.name = path
        with open(path, "rb") as f:
            self._data = f.read()
    def getvalue(self):
        return self._data

_file = DummyFile(file_path)

In [5]:
with tempfile.NamedTemporaryFile(mode='wb', delete=False) as tmp_file:
    tmp_file.write(_file.getvalue())
    tmp_file_path = tmp_file.name
#PDF 파일 업로드
if _file.name.endswith('.pdf'):
    loader = PyPDFLoader(file_path=tmp_file_path)
#Word 파일 업로드
elif _file.name.endswith('.docx'):
    loader = Docx2txtLoader(file_path=tmp_file_path)
else:
    raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 업로드해주세요.")
pages = loader.load()

In [6]:
print(pages)

[Document(metadata={'source': 'C:\\Users\\user\\AppData\\Local\\Temp\\tmpedt4i9vn'}, page_content='경력기술서\n\n[개인 정보]\n\n이메일: fourleaves8@gmail.com\n\n연락처: 010-4788-7980\n\n\n\n[전문 분야]\n\n• 데이터 사이언스       \t\t • AI 엔지니어링\t\n\n• 데이터 분석           \t\t • 프롬프트 엔지니어링\n\n• 생물분자 및 화학공학 연구\n\n\n\n[요약]\n\n저는 생명·의학 데이터를 처리하고 분석 플랫폼을 구축한 경험 및 복잡한 데이터를 효율적으로 분석해 의사결정을 지원하는 역량을 갖추고 있습니다. Python 기반의 데이터 전처리, 알고리즘 고도화, 분류 및 예측 모델 개발을 통해 항암제 신규 적응증 탐색 플랫폼을 성공적으로 구현한 바 있습니다. 또한 데이터 수집·정규화·시각화 전 과정을 주도하며, 스타트업의 코스닥 상장 및 실제 제약사 프로젝트에 기여한 실무 경험을 쌓았습니다. \n\n\n\n[보유 역량 및 기술]\n\n프로그래밍 언어\n\nPython, R, SQL, Java\n\n도구 및 프레임워크\n\npandas, NumPy, Matplotlib, seaborn, Plotly, scikit-learn, PyTorch, TensorFlow, RAG, LangChain, MySQL, Oracle\n\n운영체제\n\nLinux(Ubuntu), Windows, macOS\n\nPalantir Foundry\n\nPipeline Builder, Contour, Ontology(Action), Workshop, AIP, Dataset\n\n기타\n\nVim / Vi, Git / GitHub, Bioinformatics Tools (Gene Set Enrichment Analysis, Bismark, Metilene, lifelines, survival, survminer, lifelines)\n

## 2. 텍스트 나누기, 임베딩, 벡터DB 생성

In [7]:
# 텍스트 나누기
text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n'],
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False
)
split_docs = text_splitter.split_documents(pages)

split_docs_with_chunk_id = []
for i, doc in enumerate(split_docs):
    new_meta = doc.metadata.copy()  # 기존 metadata 보존
    new_meta['chunk_id'] = str(i)
    split_docs_with_chunk_id.append(
        Document(page_content=doc.page_content, metadata=new_meta)
    )

In [8]:
split_docs_with_chunk_id

[Document(metadata={'source': 'C:\\Users\\user\\AppData\\Local\\Temp\\tmpedt4i9vn', 'chunk_id': '0'}, page_content='경력기술서\n\n[개인 정보]\n\n이메일: fourleaves8@gmail.com\n\n연락처: 010-4788-7980\n\n\n\n[전문 분야]\n\n• 데이터 사이언스       \t\t • AI 엔지니어링\t\n\n• 데이터 분석           \t\t • 프롬프트 엔지니어링\n\n• 생물분자 및 화학공학 연구\n\n\n\n[요약]\n\n저는 생명·의학 데이터를 처리하고 분석 플랫폼을 구축한 경험 및 복잡한 데이터를 효율적으로 분석해 의사결정을 지원하는 역량을 갖추고 있습니다. Python 기반의 데이터 전처리, 알고리즘 고도화, 분류 및 예측 모델 개발을 통해 항암제 신규 적응증 탐색 플랫폼을 성공적으로 구현한 바 있습니다. 또한 데이터 수집·정규화·시각화 전 과정을 주도하며, 스타트업의 코스닥 상장 및 실제 제약사 프로젝트에 기여한 실무 경험을 쌓았습니다. \n\n\n\n[보유 역량 및 기술]\n\n프로그래밍 언어\n\nPython, R, SQL, Java\n\n도구 및 프레임워크\n\npandas, NumPy, Matplotlib, seaborn, Plotly, scikit-learn, PyTorch, TensorFlow, RAG, LangChain, MySQL, Oracle\n\n운영체제\n\nLinux(Ubuntu), Windows, macOS\n\nPalantir Foundry\n\nPipeline Builder, Contour, Ontology(Action), Workshop, AIP, Dataset\n\n기타\n\nVim / Vi, Git / GitHub, Bioinformatics Tools (Gene Set Enrichment Analysis, Bismark, Metilene, lifelines, survival, survmi

In [9]:
# OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')

# Chroma 벡터스토어 생성 (persist_directory 지정하면 재사용 가능)
persist_dir = './chroma_resume_db'
if not os.path.exists(persist_dir):
    os.makedirs(persist_dir, exist_ok=True)

# 새 vectorstore 생성
vectorstore = Chroma.from_documents(
    documents=split_docs_with_chunk_id,
    embedding=embeddings_model,
    persist_directory=None
)

## 3. Retriever 구성 (검색 옵션: simple similarity or MMR)

In [12]:
# retriever 설정
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 다양성을 고려한 검색
    search_kwargs={
        "k": 5,       # 최종 반환할 문서 수
        "fetch_k": 20  # 검색 후보로 고려할 청크 수
    }
)

query = "내가 다녔던 회사를 알아맞춰봐"
docs = retriever.invoke(query)

# 청크 내용을 하나로 합쳐서 GPT-40-mini에 전달
context = "\n\n".join([d.page_content for d in docs])

## 4. 검색 → LLM에 컨텍스트 전달 → 응답 받기

In [22]:
client = OpenAI()
response = client.chat.completions.create(
    model='gpt-4o-mini',  # 모델은 여기서 지정
    messages=[
        {'role': 'system', 'content': '다음 정보를 기반으로 사용자가 다녔던 회사를 알려주세요.'},
        {'role': 'user', 'content': context + '\n\n질문: ' + query}
    ]
)

print(response.choices[0].message.content)

당신이 다녔던 회사는 "(주)온코크로스"입니다. 이 회사에서 AI 연구소의 주임연구원으로 2021년 1월부터 2024년 12월까지 근무했으며, 다양한 데이터 사이언스 및 AI 관련 프로젝트에 참여하였다고 기술서에서 명시되어 있습니다.


In [20]:
response

ChatCompletion(id='chatcmpl-CUahczI3IDcOLsUT5TUNwbPxbmeWY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='사용자가 다녔던 회사는 "(주)온코크로스"입니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1761406960, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=19, prompt_tokens=1987, total_tokens=2006, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

## 구현

In [1]:
import os
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import requests
from bs4 import BeautifulSoup

In [14]:
# 1. API KEY
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

st.title("💬 자기소개서 작성 도우미")

# 2. 경력기술서 업로드
uploaded_file = st.file_uploader("경력기술서 업로드 (.pdf/.docx)", type=['pdf','docx'])
if uploaded_file:
    if uploaded_file.name.endswith('.pdf'):
        loader = PyPDFLoader(uploaded_file)
    else:
        loader = Docx2txtLoader(uploaded_file)
    docs = loader.load_and_split()

2025-10-26 00:41:11.390 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 00:41:11.407 
  command:

    streamlit run c:\Users\user\miniconda3\envs\ragenv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-26 00:41:11.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 00:41:11.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 00:41:11.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 00:41:11.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 00:41:11.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-26 00:41:11.409 Thread 'Mai